In [2]:
%pylab inline
import numpy as np 

Populating the interactive namespace from numpy and matplotlib


# Anomaly De

## 기본적인 구조

의학 이미지안에서 질병등을 발견하는 것은 매우 중요합니다. <br>
Anomaly Detection GAN은 Discriminator를 통해서 의학 이미지 상에서 질병을 찾아내도록 도와줍니다. <br>
물론 Anomaly라는 개념하에서 반드시 질병일 필요는 없습니다. 


기본적인 구조는 아래와 같으며, Deep Convolutional Generative Adversarial Network (DCGAN)을 사용했으며, <br>
오른쪽 그림은 t-SNE embedding으로 파란색은 정상, 빨간색은 비정상(anomalous images) 를 나타냅니다.

<img src="./images/anogan.png" class="img-responsive img-rounded">

* Medical Images: $ \mathbf{I}_m $ 으로서 $ m = 1,2,..., M $ 이며, 모두 정상적인 상태의 (질병이 없는) 이미지, $ \mathbf{I}_m  \in \mathbb{R}^{a*b}$ 입니다.

**학습**시에는 오직 $ \mathbf{I}_m $ (질병없는 정상 이미지)가 주어지며, 해당 이미지로부터 c x c 크기의 image patch $ x_{k,m} $ 를 랜덤으로 위치를 샘플링해서 추출하게 됩니다.<br>
즉, $ \mathbf{x} = x_{k,m} \in \mathcal{X} $ 이며, 쉽게 설명하면 큰 메디컬 사진안에서 c x c 크기의 일부구간을 랜덤으로 잘라내고, 잘라진 구간을 비젼에서는 image patch 라고 말하며, 해당 image patch 는 질병이 없는 (위의 파란색) manifold  $ \mathcal{X} $ 를 학습한다는 뜻입니다. 

테스트시에는 $ \mathbf{y}_n, l_n $ 이 주어지며, $ \mathbf{y}_n $ 은 testing data $ \mathbf{J} $ 의 c x c 싸이즈의 이미지입니다.<br>
$ l_n \in \{0, 1\} $ 은 바이너리 값으로서 레이블값이라고 생각하면 되며, 오직 테스트를 할때만 정말 잘 되는지 판단하기 위해서 주어지는 값입니다.

## ??

기본적인 

## Dataset

http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm